In [ ]:
from operetta_compose.tasks.harmony_to_ome_zarr import _parse_harmony_index
from pathlib import Path


well_df, img_df = _parse_harmony_index(Path("tests/operetta_plate/Images/"))

In [ ]:
img_df

In [ ]:
img_df.groupby(["row", "col"]).agg(list)

In [ ]:

import numpy as np
import tifffile


class TiffLoader:
    """Png loader for tiled images."""

    def __init__(self, dir: str, paths: list[str]):
        """Initialize the PngLoader.

        Args:
            dir (str): Directory where the PNG files are located.
            paths (list[str]): List of PNG file paths relative to the directory.
        """
        self.dir = dir
        self.paths = paths

    def load(self) -> np.ndarray:
        """Load the PNG files and stack them into a numpy array."""
        arrays = []
        for path in self.paths:
            full_path = f"{self.dir}/{path}"
            img = tifffile.imread(full_path)
            arrays.append(np.array(img))
        stacked_arrays = np.stack(arrays, axis=0)[None, None, :, :, :]
        return stacked_arrays

    @property
    def dtype(self):
        """Return the data type of the loaded images."""
        return "uint8"

In [ ]:
from fractal_converters_tools import PlatePathBuilder, TiledImage
import numpy as np
from ngio import PixelSize

from fractal_converters_tools import OriginDict, Point, Tile

alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

tiled_images = []
for image in img_df.groupby(["row", "col"]).agg(list).itertuples():
    print(image.Index)
    tiled_image = TiledImage(
        name="None",
        path_builder=PlatePathBuilder(
                plate_name="operetta_plate",
                row=alphabet[image.Index[0] - 1],
                column=image.Index[1],
                acquisition_id=0,  
            ),
        channel_names=[image.ChannelName[0]],
    )
    
    # filter the img_df to only have row == image.Index[0] and col == image.Index[1]
    img_subset = img_df[(img_df["row"] == image.Index[0]) & (img_df["col"] == image.Index[1])]
    for tile_df in img_subset.itertuples():
        pixel_size = PixelSize(
            x=tile_df.res_x,
            y= tile_df.res_y,
            z=tile_df.res_z,
            t=1
        )
        
        top_left = Point(
                x=tile_df.pos_x,
                y=tile_df.pos_y,
                z=0, # to be refined with the z position
                t=0,  # to be refined with the t position
                c=tile_df.channel - 1, # channel is 1-indexed in the dataframe, but 0-indexed in the Point object
            )
        
        # let's fix the len_x to avoid off-by-one errors
        len_x = 1080 * pixel_size.x
        len_y = 1080 * pixel_size.y
        print(f"len_x: {len_x}, len_y: {len_y}")
        print(img_df.len_x, img_df.len_y)
        bottom_right = Point(
            x=tile_df.pos_x + len_x,
            y=tile_df.pos_y + len_y,
            z=tile_df.len_z,  # to be refined with the z position
            c=tile_df.channel,  # channel is 1-indexed in the dataframe, but 0-indexed in the Point object
            t=1,  # to be refined with the t position
        )
        
        origin = OriginDict(
                x_micrometer_original=tile_df.pos_x,
                y_micrometer_original=tile_df.pos_y,
                z_micrometer_original=0,
                t_original=0,
            )
        
        loader = TiffLoader(
            dir="tests/operetta_plate/Images/",
            paths=[tile_df.img_name]
            )
        
        tile_obj = Tile.from_points(
            top_l=top_left,
            bot_r=bottom_right,
            pixel_size=PixelSize(
                x=pixel_size.x, y=pixel_size.y, z=pixel_size.z, t=pixel_size.t
            ),
            origin=origin,
            data_loader=loader,
        )
        tiled_image.add_tile(tile_obj)
    tiled_images.append(tiled_image)

In [ ]:
from fractal_converters_tools import (
    AdvancedComputeOptions,
    build_parallelization_list,
    initiate_ome_zarr_plates,
)

# Build parallelization list for the compute phase
parallelization_list = build_parallelization_list(
    zarr_dir="./test_zarr",
    tiled_images=tiled_images,
    overwrite=True,
    advanced_compute_options=AdvancedComputeOptions(),
)

# Initiate the OME-Zarr plates with the tiled image
initiate_ome_zarr_plates(
    zarr_dir="./test_zarr",
    tiled_images=tiled_images,
    overwrite=True,
)

In [ ]:
from fractal_converters_tools import ConvertParallelInitArgs, generic_compute_task

for task in parallelization_list:
    print(f"Running task: {task}")
    generic_compute_task(
        zarr_url=task["zarr_url"],
        init_args=ConvertParallelInitArgs(**task["init_args"]),
    )